In [1]:
import wrds

import pandas as pd
import numpy as np

from ff_category import FFCategory

pd.options.mode.chained_assignment = None

In [2]:
# Initialize connection
db = wrds.Connection(wrds_username='tadej')

Loading library list...
Done


# Compustat

Here I get the required compustat data.

## Annual fundamentals

### Download

I get the following rows from `funda` table in the `compd` database:
- `gvkey, fyear, fic, sich, csho, prcc_f, at, lt, pstk, dltt`
- `dlc, act, ppegt, dvt, prstkc, oiadp, txt, xint, che, ppent, capx, gdwl`
- `naicsh`

The only filtering I do is to throw out things without a year (happens sometimes), and apply the standard, criptic data selection that prevents duplicates.

In [20]:
# SQL select string
select_str = '''
SELECT gvkey, fyear, fic, sich, csho, prcc_f, at, lt, pstk, dltt, dlc, 
       act, ppegt, dvt, prstkc, oiadp, txt, xint, che, ppent, capx, gdwl,
       naicsh
FROM compd.funda 
WHERE indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
        AND fyear IS NOT NULL
'''

# Get data
data_fa = db.raw_sql(select_str)

# Still there are, for some reason, 4 duplicates: remove them
data_fa.drop_duplicates(subset = ['fyear', 'gvkey'], inplace = True)

### New variables

Here I create the following new variables:
- `me`: Market value of equity, obtained by multiplying common shares outsdanding with their price 
- `be`: Book value of equity, equal to total assets minus liabilities minus equity (some measure of it)
- `bliab`: Book value of liabilities: assets minus equity
- `mv`: Market value: Market value of equity + assets - book equity
- `blev`: Leverage: book value of libilities divided by assets
- `q`: Tobin's q (market value of the firm divided by assets)
- `q2`: Alternative Tobin's q
- `paya`: Payouts over assets
- `bba`: Buybacks over assets
- `os`: Net operating surplus
- `payos`: Payouts over operating surplus
- `bbos`: Buybacks over operating surplus
- `ca`: Cash holdings over assets

In [21]:
data_fa.eval('''
    me = csho*prcc_f 
    be = at - lt - pstk 
    bliab = at - be
    blev = bliab/at
    mv = me + at - be
    q = mv/at
    q2 = (me+dltt+dlc-act)/ppegt
    paya = (dvt + prstkc)/ at
    bba = prstkc / at
    os = oiadp - txt - xint  
    payos = (dvt + prstkc)/ os
    bbos = prstkc/ os
    ca = che / at''', inplace = True)

## Peters and Taylor Q

Here I download the Peters and Taylor (improved) Q measure, that is the `total_q` table in the `totalq` database.

I download the variables `gvkey`, `fyear`, `q_tot`, `k_int` and `k_int_offbs`.

In [22]:
# SQL select string
select_str = '''
SELECT fyear, gvkey, q_tot, k_int, k_int_offbs
FROM totalq.total_q
'''

# Get data
data_q = db.raw_sql(select_str)

## Ratings data

I download the data from Compustat, specifically, I use the Ratings table under North America daily. I then apply a recoding of the ratings, as shown below.

**Warning**: That dataset is obsolete, it has not been updated since 2017. There is a newer credit ratings dataset (under Capital Q), however, based on my investigation, it has much less data than the old one (and some data it has is ambiguous), so it is not a suitable substitute here.

In [23]:
select_str = '''
SELECT date_part('year', datadate) AS year, splticrm, gvkey
FROM compd.adsprate
WHERE date_part('month', datadate) = 12
AND splticrm IS NOT NULL
AND splticrm NOT IN ('N.M.', 'Suspended', 'SD')
'''

data_r = db.raw_sql(select_str)

# Numerify the ratings
replace_dict = {
    'AAA': 1, 'AA': 2, 'A': 3, 'BBB': 4, 'BB': 5, 'B': 6,
    'CCC': 7, 'CC': 8, 'C': 9, 'D': 10
}

data_r['sprating'] = data_r['splticrm']

# The order is important here
for key, value in replace_dict.items():
    data_r.loc[data_r.sprating.str.contains(key), 'sprating'] = str(value)
    
data_r['sprating'] = data_r['sprating'].astype(int) 

# Create some rating booleans
data_r['AAtoAAA'] = data_r.sprating <= 2
data_r['BBBtoA'] = (data_r.sprating <= 4) & (data_r.sprating >= 3)

# Drop some variables
data_r.drop(columns = ['splticrm'], inplace = True)

## Merging and minor calculations

Here I merge the three datasets downloaded so far, and compute some new variables:
- `k_pt`: A measure of capital, basically adding physical and intangible capital together
- `shareintant`: Share of intangible capital in the above measure

After that, I sort data by `gvkey` and `year` and add cummulative count for `gvkey` : this will be the age of the firm (assuming it appears in the dataset every year of its existence). I then log this to produce `logage`.

In [24]:
# Merge fundamentals annual and Q data
data = data_fa.merge(data_q, on = ['gvkey', 'fyear'], how = 'left')

# Rename fyear to year and then merge
data.rename(columns = {'fyear': 'year'}, inplace = True)
data = data.merge(data_r, on = ['gvkey', 'year'], how = 'left')

# New variables
data.eval('''
    k_pt = ppent + k_int
    shareintan = k_int / k_pt''', inplace = True)

# Sort and age
data.sort_values(['gvkey', 'year'], inplace = True)
data['logage'] = np.log(data.groupby('gvkey').cumcount() + 1)

## Aggregates pre-filtering

Here I compute some aggregates (by year), which are later used to produce a graph. These aggregates are:
- `a_capx_all_preEx`: sum of `capx`
- `a_capx_US_preEx`: sum of `capx`, only for US firms
- `a_pay_preEx`: sum of `dvt` and `prstkc`
- `a_prstkc_preEx`: sum of `prstkc`
- `a_at_preEx`: sum of `at`
- `a_paya_preEx = a_pay_preEx / a_at_preEx`
- `a_bba_preEx = a_prstkc_preEx / a_at_preEx`

**Note**: Check the thing about goodwill later

In [25]:
# Compute aggregate variables from data
data_agg = data.groupby('year').apply(lambda x: pd.Series({
    'a_capx_all_preEx': x['capx'].sum(),
    'a_capx_US_preEx': x.query('fic == "USA"')['capx'].sum(),
    'a_pay_preEx': x.eval('dvt + prstkc').sum(),
    'a_prstkc_preEx': x['prstkc'].sum(),
    'a_at_preEx': x['at'].sum()
}))

# Add some composite aggregate variables
data_agg.eval('''
    a_paya_preEx = a_pay_preEx / a_at_preEx
    a_bba_preEx = a_prstkc_preEx / a_at_preEx''', inplace = True)

## Filtering and aggregates

Finally, I do some filtering:
1. Drop all entries where `at`, `gvkey`, `be`, `me`, `bliab` or `q` are NA, as well as those where `at` is smaller than 1 (million), and where either `be` or `me` are negative.
2. Drop all entries with years smaller or equal to 1961, and drop a peculiar case with `gvkey` 4828 and `year` 2001.
3. Drop all entries with `sich` in \[4900, 4999\] (utilities), \[6000, 6999\] (financials) or \[5300, 5399\] (real estate)
4. Keep only US companies (`fic = USA`)

At this point I also compute some aggregates (I do this right after step 3).

Finally, I assign the Fama-French categories to the SIC numbers (based on my own implementation).

In [26]:
# Drop NAs, filtering
data.dropna(subset = ['year', 'gvkey', 'be', 'me', 'bliab', 'q'], inplace = True)
data.query('at >= 1 & be > 0 & me > 0 & year > 1961 & ~(gvkey == 4828 & year == 2001)', inplace = True)

# Exclude some sic codes
data.query('~sich.between(4900, 4999) & ~sich.between(6000, 6999) & ~sich.between(5300, 5399)', inplace = True)

# Some additional aggregates
k = data.groupby('year').apply(lambda x: pd.Series({
    'a_capx_all_wEx': x['capx'].sum(),
    'a_capx_US_wEx': x.query('fic == "USA"')['capx'].sum(),
}))

data_agg[['a_capx_all_wEx', 'a_capx_US_wEx']] = k

# Drop non-USA
data.query('fic == "USA"', inplace = True)

# Get the 10 category FF classification
ff = FFCategory('https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Siccodes10.zip')
data['ff10'] = ff.assign_ff(data.sich)

## Table 10

Here I reproduce the table 10 from the paper, which shows quantiles for different Q measures over two time periods: 1975 - 1980 and 2010 - 2015.

As you can see in the table bellow (and by comparing it with the one from the paper), we have almost the exact same numbers, except for extreme quantiles.

In [39]:
# Quantiles to get
quantiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
    
# Get quantiles
q1 = data.query('year.between(1975, 1980)')[['q', 'q2', 'q_tot']].quantile(quantiles).round(1)
q2 = data.query('year.between(2010, 2015)')[['q', 'q2', 'q_tot']].quantile(quantiles).round(1)

# Joint table
qs_table = pd.concat([q1, q2], keys = ['1975-1980', '2010-2015'], axis = 1)

# Save to tables
qs_table.to_csv('Tables/Table10.csv')

# Show results
qs_table

1975-1980             2010-2015                
             q    q2 q_tot         q        q2 q_tot
0.01       0.5  -5.5  -2.8       0.5      -6.4  -0.9
0.05       0.7  -1.5  -0.5       0.7      -0.6  -0.2
0.10       0.7  -0.7  -0.3       0.9       0.0   0.0
0.25       0.8  -0.1  -0.1       1.1       0.7   0.3
0.50       1.0   0.3   0.2       1.6       2.1   0.8
0.75       1.2   0.8   0.6       2.5       8.0   1.5
0.90       1.8   2.4   1.4       4.4      33.2   3.4
0.95       2.5   4.6   2.6       6.4      97.4   5.9
0.99       6.5  31.3  14.9      16.5  107482.7  24.9

## Segmentation mapping

Here I create some segmentation variables (mostly NAICS related).

### NAICS 3

I create a new variable `naics3` from the first 3 digits of `naicsh`. Where this is missing, I do the following assignment:
- For each sic and naics pair I count the number of entries appearing in it.
- For each sic, I take the highest NAICS (from the previous step)
- When possible, I match this naics to corresponding SIC

Additionally, I also manually adjust some naics values.

In [48]:
# Naics to string
data['naics_str'] = data.naicsh.astype(str).str.slice(0, -2)
data.loc[data.naicsh.isna(), 'naics_str'] = np.nan

# Default NAICS3
data['naics3'] = data.loc[data.naics_str.str.len() >= 3, 'naics_str'].str.slice(0,3)
data['naics3'] = data['naics3'].astype(float)

# Create the matching dictionary
agg = data.groupby(['sich', 'naics3']).size()
match_dict = agg.groupby(level = 0, group_keys = False).nlargest(1)
match_dict = match_dict.reset_index().drop(columns= [0])

# Reset index before merging
data = data.reset_index(drop=True)

# Match
naics3_matched = data[['sich']].merge(match_dict, on = 'sich', how = 'left')['naics3']
data.loc[data.naics3.isna(), 'naics3'] = naics3_matched

# Special matching
data.loc[data.naics3.isna() & (data.sich == 3412), 'naics3'] = 332

### BEA Codes

Here I map to BEA codes and segments, using the provided excel and stata files. After that I merge with the computed BEA file, using the industry depreciation.

In [60]:
# Read in files
bea_codes = pd.read_excel('Data/User inputs/NAICS2BEA.xlsx').rename(columns={'naics': 'naics3'})
bea_segments = pd.read_stata('Data/Temp/levelkey.dta').rename(columns={'ind_short': 'indcode'})

# Perform the merges (they have to be done in sequence)
data = data.merge(bea_codes, how = 'left')
data = data.merge(bea_segments, how = 'left')



In [63]:
data

,gvkey,year,fic,sich,csho,prcc_f,at,lt,pstk,dltt,...,BBBtoA,k_pt,shareintan,logage,ff10,naics_str,naics3,beacode,indcode,siccode
0,001000,1970.0,USA,NaN,2.446,10.000000,33.450,22.906,0.000,0.917,...,NaN,14.431471,0.384955,2.302585,NaN,NaN,NaN,NaN,NaN,NaN
1,001000,1971.0,USA,NaN,2.995,5.750000,29.330,20.948,0.000,10.318,...,NaN,15.445105,0.505410,2.397895,NaN,NaN,NaN,NaN,NaN,NaN
2,001000,1972.0,USA,NaN,2.902,5.125000,19.907,12.886,0.000,7.000,...,NaN,15.733992,0.554277,2.484907,NaN,NaN,NaN,NaN,NaN,NaN
3,001000,1973.0,USA,NaN,2.840,1.750000,21.771,13.204,0.000,7.000,...,NaN,17.388598,0.573974,2.564949,NaN,NaN,NaN,NaN,NaN,NaN
4,001000,1974.0,USA,NaN,2.150,2.125000,25.638,15.381,0.414,7.000,...,NaN,17.254430,0.670172,2.639057,NaN,NaN,NaN,NaN,NaN,NaN
5,001000,1975.0,USA,NaN,2.098,4.375000,23.905,11.596,2.069,4.592,...,NaN,20.082090,0.638334,2.708050,NaN,NaN,NaN,NaN,NaN,NaN
6,001000,1976.0,USA,NaN,2.207,5.750000,38.586,22.911,1.569,13.583,...,NaN,31.006240,0.469784,2.772589,NaN,NaN,NaN,NaN,NaN,NaN
7,001000,1977.0,USA,NaN,2.226,9.250000,44.025,28.149,0.000,18.116,...,NaN,36.817130,0.459029,2.833213,NaN,NaN,NaN,NaN,NaN,NaN
8,001001,1983.0,USA,NaN,3.568,7.250000,14.080,6.257,0.000,4.344,...,NaN,20.686280,0.587359,1.791759,NaN,NaN,NaN,NaN,NaN,NaN
9,001001,1984.0,USA,NaN,3.568,3.750000,16.267,7.305,0.000,4.181,...,NaN,29.168240,0.564081,1.945910,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
data[['naics3']].merge(bea_codes, how = 'left')

,naics3,beacode
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
